In [1]:
import os
import pandas as pd
os.chdir('..')
from src.models.entmoot_functions import *
from src.models.train_animation_predictor import *

In [2]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

### 1. Load and preprocess training data for optimizer fitting

#### Load data

In [15]:
X_train = pd.read_csv('data/model_1/model_1_train.csv')

In [16]:
X_train.head()

,filename,animation_id,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,stroke_width,opacity,stroke_opacity,fill_r,fill_g,fill_b,stroke_r,stroke_g,stroke_b,svg_fill_r,diff_fill_r,svg_fill_g,diff_fill_g,svg_fill_b,diff_fill_b,svg_stroke_r,diff_stroke_r,svg_stroke_g,diff_stroke_g,svg_stroke_b,diff_stroke_b,rel_width,rel_height,rel_x_position,rel_y_position,nr_paths_svg
0,logo_0,1,13.535693,6.971131,-0.009867,-0.915823,-2.984741,5.383109,0.495111,-2.146576,4.868704,-0.944599,0.0,1.0,1.0,197,0,123,0,0,0,238.6,-41.6,162.6,-162.6,127.4,-4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.400180,0.472019,0.700090,0.564323,10
1,logo_0,0,-1.000982,4.641413,1.455743,-0.487705,-1.037431,6.984421,-0.485484,-3.933893,6.030271,-1.227060,0.0,1.0,1.0,250,187,0,0,0,0,238.6,11.4,162.6,24.4,127.4,-127.4,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.671553,0.250000,0.335776,10
2,logo_0,2,10.590673,0.533370,8.743198,0.241546,-2.132272,0.620374,1.687153,6.592582,3.034471,-1.564105,0.0,1.0,1.0,236,116,4,0,0,0,238.6,-2.6,162.6,-46.6,127.4,-123.4,0.0,0.0,0.0,0.0,0.0,0.0,0.787194,0.328324,0.499991,0.835838,10
3,logo_0,3,4.222565,-0.735711,5.308626,-0.090910,-4.875907,2.410124,0.314957,-1.771255,3.811236,-0.731849,0.0,1.0,1.0,197,0,123,0,0,0,238.6,-41.6,162.6,-162.6,127.4,-4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.400206,0.472019,0.299897,0.564323,10
4,logo_0,4,1.776277,8.866785,-2.770646,-0.955766,-4.725605,5.412009,0.597616,2.331442,2.209505,-1.606922,0.0,1.0,1.0,236,116,4,0,0,0,238.6,-2.6,162.6,-46.6,127.4,-123.4,0.0,0.0,0.0,0.0,0.0,0.0,0.586900,0.315475,0.500106,0.170381,10


In [17]:
X_train = retrieve_m1_predictions(X_train)
X_train = retrieve_animation_midpoints(X_train, drop=True)

2021-04-22 11:50:26::INFO::Number of animated paths: 2141/8921
2021-04-22 11:50:26::INFO::Non-animated paths won't be used for training
2021-04-22 11:50:26::INFO::Start extraction midpoint of animated paths as feature
data/initial_svgs/logo_253.svg, animation ID 24: svg2path fails and path bbox cannot be computed. list index out of range
data/initial_svgs/logo_253.svg: svg2path fails and bbox of multiple paths cannot be computed. list index out of range
data/initial_svgs/logo_253.svg, animation_id 24, animated_animation_ids [24, 25, 27, 26]: rel_x_position not defined and set to 0.5. float division by zero
data/initial_svgs/logo_253.svg, animation_id 24, animated_animation_ids [24, 25, 27, 26]: rel_y_position not defined and set to 0.5. float division by zero
data/initial_svgs/logo_253.svg, animation ID 25: svg2path fails and path bbox cannot be computed. list index out of range
data/initial_svgs/logo_253.svg: svg2path fails and bbox of multiple paths cannot be computed. list index out

In [18]:
scaler = pickle.load(open(config.scaler_path, 'rb'))
X_train[config.sm_features] = scaler.transform(X_train[config.sm_features])

In [19]:
X_train.head()

,filename,animation_id,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,stroke_width,opacity,stroke_opacity,fill_r,fill_g,fill_b,stroke_r,stroke_g,stroke_b,svg_fill_r,diff_fill_r,svg_fill_g,diff_fill_g,svg_fill_b,diff_fill_b,svg_stroke_r,diff_stroke_r,svg_stroke_g,diff_stroke_g,svg_stroke_b,diff_stroke_b,rel_width,rel_height,rel_x_position,rel_y_position,nr_paths_svg,animated,animated_animation_ids,rel_position_to_animations,rel_x_position_to_animations,rel_y_position_to_animations
0,logo_0,1,1.100396,1.053341,0.166888,-0.302653,-0.574280,1.332109,0.058484,-1.002773,2.873231,-0.802108,0.0,1.0,1.0,0.428841,-1.471855,0.099901,0,0,0,1.596318,-0.812013,0.763574,-2.243770,0.279142,-0.120576,0.0,0.0,0.0,0.0,0.0,0.0,0.636605,0.492414,1.253294,0.716467,-0.745455,1,"[1, 0, 2, 3, 4, 5]","(0.7000899895197011, 0.5643229197828795)",0.790540,0.311598
1,logo_0,0,-0.734658,0.648019,0.454550,-0.217326,0.130276,1.941491,-0.566684,-1.797855,3.556063,-1.035641,0.0,1.0,1.0,0.940915,0.672469,-1.249679,0,0,0,1.596318,-0.251789,0.763574,0.110768,0.279142,-1.707396,0.0,0.0,0.0,0.0,0.0,0.0,1.069430,1.220115,-0.651467,-0.420590,-0.745455,1,"[1, 0, 2, 3, 4, 5]","(0.25, 0.33577636718263004)",-0.995922,-0.694049
2,logo_0,2,0.728628,-0.066694,1.884896,-0.071980,-0.265848,-0.480356,0.818459,2.884815,1.794969,-1.314303,0.0,1.0,1.0,0.805650,-0.141686,-1.205790,0,0,0,1.596318,-0.399773,0.763574,-0.783201,0.279142,-1.655792,0.0,0.0,0.0,0.0,0.0,0.0,2.314716,-0.031644,0.406484,2.067299,-0.745455,1,"[1, 0, 2, 3, 4, 5]","(0.4999911632674701, 0.8358382216582674)",-0.003677,1.506316
3,logo_0,3,-0.075257,-0.287487,1.210775,-0.138241,-1.258523,0.200736,-0.056371,-0.835812,2.251594,-0.626210,0.0,1.0,1.0,0.428841,-1.471855,0.099901,0,0,0,1.596318,-0.812013,0.763574,-2.243770,0.279142,-0.120576,0.0,0.0,0.0,0.0,0.0,0.0,0.636719,0.492414,-0.440306,0.716467,-0.745455,1,"[1, 0, 2, 3, 4, 5]","(0.2998969019667724, 0.5643229197828794)",-0.797875,0.311598
4,logo_0,4,-0.384067,1.383145,-0.374984,-0.310614,-1.204143,1.343107,0.123835,0.989260,1.310010,-1.349703,0.0,1.0,1.0,0.805650,-0.141686,-1.205790,0,0,0,1.596318,-0.399773,0.763574,-0.783201,0.279142,-1.655792,0.0,0.0,0.0,0.0,0.0,0.0,1.446234,-0.078501,0.406970,-1.243457,-0.745455,1,"[1, 0, 2, 3, 4, 5]","(0.5001060407903573, 0.1703812636898346)",-0.003221,-1.421818


#### Merge path animation vectors and labels

In [20]:
with open('data/surrogate_model/sm_animation_vectors_label.pkl', 'rb') as f:
    animations = pickle.load(f)

In [21]:
animations.head()

,file,animation_id,order_id,path_prob,begin_value,model_output,animated_animation_ids,animated_order_ids,backend_mapping,logo_id,animation_number,alias,animation_file,time,label
0,logo_316_animation_0,4,1,0.2,1.00,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8...","[4, 24, 23]","[1, 6, 7]","[0, 1, 0, 0, 0, 0, 1, 1]",316,0,Jakob,animation/logo_316_animation_0.svg,"Timestamp(seconds=1617783014, nanoseconds=3260...",Good
1,logo_316_animation_0,24,6,0.6,1.25,"[0, 0, 0, 0, 0, 1, -1.0, -1.0, -1.0, -1.0, -1....","[4, 24, 23]","[1, 6, 7]","[0, 1, 0, 0, 0, 0, 1, 1]",316,0,Jakob,animation/logo_316_animation_0.svg,"Timestamp(seconds=1617783014, nanoseconds=3260...",Good
2,logo_316_animation_0,23,7,0.2,1.50,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....","[4, 24, 23]","[1, 6, 7]","[0, 1, 0, 0, 0, 0, 1, 1]",316,0,Jakob,animation/logo_316_animation_0.svg,"Timestamp(seconds=1617783014, nanoseconds=3260...",Good
3,logo_316_animation_1,4,1,0.2,1.00,"[1, 0, 0, 0, 0, 0, 0.13436424411240122, 0.8474...","[4, 24, 23]","[1, 6, 7]","[0, 1, 0, 0, 0, 0, 1, 1]",316,1,Jonathan,animation/logo_316_animation_1.svg,"Timestamp(seconds=1617813571, nanoseconds=9700...",Very Bad
4,logo_316_animation_1,24,6,0.6,1.25,"[0, 0, 1, 0, 0, 0, -1.0, -1.0, -1.0, 0.7637746...","[4, 24, 23]","[1, 6, 7]","[0, 1, 0, 0, 0, 0, 1, 1]",316,1,Jonathan,animation/logo_316_animation_1.svg,"Timestamp(seconds=1617813571, nanoseconds=9700...",Okay


In [22]:
# get correct filenames to match
animations['filename'] = animations['file'].apply(lambda row: row.split("_animation")[0])

In [23]:
# drop unnecessary columns
animations.drop(['file','order_id','path_prob','begin_value','animated_animation_ids','animated_order_ids','backend_mapping','logo_id','animation_number','alias','animation_file', 'time'], axis=1, inplace=True)

# map label names to label indexes and delete entries without rating
mapping = {'Very Bad': 0, 'Bad': 1, 'Okay': 2, 'Good': 3, 'Very Good': 4}
animations.replace({'label': mapping}, inplace=True)
animations = animations[animations['label'] != 'no_rating']
animations.reset_index(drop=True, inplace=True)

In [24]:
animations.head()

,animation_id,model_output,label,filename
0,4,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8...",3,logo_316
1,24,"[0, 0, 0, 0, 0, 1, -1.0, -1.0, -1.0, -1.0, -1....",3,logo_316
2,23,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....",3,logo_316
3,4,"[1, 0, 0, 0, 0, 0, 0.13436424411240122, 0.8474...",0,logo_316
4,24,"[0, 0, 1, 0, 0, 0, -1.0, -1.0, -1.0, 0.7637746...",2,logo_316


In [25]:
initial_data = animations.merge(X_train, how='left', on=['filename', 'animation_id'])

In [26]:
print(f"Before: Train: {initial_data.shape}.")
initial_data.dropna(inplace=True)
print(f"After: Train: {initial_data.shape}.")

Before: Train: (23481, 45).
After: Train: (4664, 45).


In [27]:
# Drop unnecessary columns
initial_data.drop(['stroke_width','opacity','stroke_opacity','stroke_r','stroke_g','stroke_b','svg_stroke_r','diff_stroke_r','svg_stroke_g','diff_stroke_g','svg_stroke_b','diff_stroke_b'], axis=1, inplace=True)

In [28]:
initial_data.head()

,animation_id,model_output,label,filename,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,fill_r,fill_g,fill_b,svg_fill_r,diff_fill_r,svg_fill_g,diff_fill_g,svg_fill_b,diff_fill_b,rel_width,rel_height,rel_x_position,rel_y_position,nr_paths_svg,animated,animated_animation_ids,rel_position_to_animations,rel_x_position_to_animations,rel_y_position_to_animations
0,4,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8...",3,logo_316,-2.160263,0.445070,-0.368192,0.210312,0.838155,-1.017779,2.181343,-0.352907,-0.039019,-0.690817,-0.894821,-0.783836,-0.591347,-0.696082,-0.420297,-0.738528,-0.253640,-0.671822,-0.122403,-0.861189,-0.921821,-1.566257,-1.881579,0.035163,1.0,"[20, 19, 23, 21, 24, 22, 4]","(0.03383765572036357, 0.04211967340054534)",-1.853896,-1.986193
1,24,"[0, 0, 0, 0, 0, 1, -1.0, -1.0, -1.0, -1.0, -1....",3,logo_316,0.354542,0.637739,1.558871,-1.288921,-0.318990,1.667101,0.432802,-0.195713,-0.346153,-0.009741,0.138988,0.443130,0.582677,-0.696082,0.710721,-0.738528,1.093609,-0.671822,1.258001,0.618456,0.094413,0.412917,0.790924,0.035163,1.0,"[20, 19, 23, 21, 24, 22, 4]","(0.5015112311156477, 0.5792887719245869)",0.002357,0.377451
2,23,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....",3,logo_316,0.490803,0.501253,1.752732,-1.265763,-0.209128,1.706432,0.509081,-0.200540,-0.257648,0.023685,0.138988,0.443130,0.582677,-0.696082,0.710721,-0.738528,1.093609,-0.671822,1.258001,0.618456,0.094472,0.412917,1.082721,0.035163,1.0,"[20, 19, 23, 21, 24, 22, 4]","(0.5015112311156477, 0.6379395431595825)",0.002357,0.635525
3,4,"[1, 0, 0, 0, 0, 0, 0.13436424411240122, 0.8474...",0,logo_316,-2.160263,0.445070,-0.368192,0.210312,0.838155,-1.017779,2.181343,-0.352907,-0.039019,-0.690817,-0.894821,-0.783836,-0.591347,-0.696082,-0.420297,-0.738528,-0.253640,-0.671822,-0.122403,-0.861189,-0.921821,-1.566257,-1.881579,0.035163,1.0,"[20, 19, 23, 21, 24, 22, 4]","(0.03383765572036357, 0.04211967340054534)",-1.853896,-1.986193
4,24,"[0, 0, 1, 0, 0, 0, -1.0, -1.0, -1.0, 0.7637746...",2,logo_316,0.354542,0.637739,1.558871,-1.288921,-0.318990,1.667101,0.432802,-0.195713,-0.346153,-0.009741,0.138988,0.443130,0.582677,-0.696082,0.710721,-0.738528,1.093609,-0.671822,1.258001,0.618456,0.094413,0.412917,0.790924,0.035163,1.0,"[20, 19, 23, 21, 24, 22, 4]","(0.5015112311156477, 0.5792887719245869)",0.002357,0.377451


#### Transform animation vector into multiple dataframe columns and change column ordering

In [29]:
initial_data[[f'an_vec_{i}' for i in range(12)]] = pd.DataFrame(initial_data['model_output'].tolist(), index=initial_data.index)

initial_data.drop(['model_output','animation_id','filename'], inplace=True, axis=1)

In [30]:
col_order = [f'an_vec_{i}' for i in range(12)] + [f'emb_{i}' for i in range(10)] + ['_'.join(['fill', ch]) for ch in ['r','g','b']] + ['_'.join(['svg_fill', ch]) for ch in ['r','g','b']] + ['_'.join(['diff_fill', ch]) for ch in ['r','g','b']] + ['rel_height','rel_width','rel_x_position','rel_y_position','rel_x_position_to_animations','rel_y_position_to_animations','nr_paths_svg','label']
initial_data = initial_data[col_order]

#### Encode labels into 4-binary labels

In [31]:
initial_data[[f'rating_{i}' for i in range(4)]] = pd.DataFrame(encode_classes(np.array(initial_data['label'])), index=initial_data.index)

initial_data.drop(['label'], axis=1, inplace=True)

initial_data.reset_index(drop=True, inplace=True)

In [32]:
initial_data.head()

,an_vec_0,an_vec_1,an_vec_2,an_vec_3,an_vec_4,an_vec_5,an_vec_6,an_vec_7,an_vec_8,an_vec_9,an_vec_10,an_vec_11,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,fill_r,fill_g,fill_b,svg_fill_r,svg_fill_g,svg_fill_b,diff_fill_r,diff_fill_g,diff_fill_b,rel_height,rel_width,rel_x_position,rel_y_position,rel_x_position_to_animations,rel_y_position_to_animations,nr_paths_svg,rating_0,rating_1,rating_2,rating_3
0,0,0,0,1,0,0,-1.000000,-1.000000,-1.0,-1.000000,0.844422,0.757954,-2.160263,0.445070,-0.368192,0.210312,0.838155,-1.017779,2.181343,-0.352907,-0.039019,-0.690817,-0.894821,-0.783836,-0.591347,-0.696082,-0.738528,-0.671822,-0.420297,-0.253640,-0.122403,-0.921821,-0.861189,-1.566257,-1.881579,-1.853896,-1.986193,0.035163,1,1,1,0
1,0,0,0,0,0,1,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,0.354542,0.637739,1.558871,-1.288921,-0.318990,1.667101,0.432802,-0.195713,-0.346153,-0.009741,0.138988,0.443130,0.582677,-0.696082,-0.738528,-0.671822,0.710721,1.093609,1.258001,0.094413,0.618456,0.412917,0.790924,0.002357,0.377451,0.035163,1,1,1,0
2,0,0,0,0,1,0,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,0.490803,0.501253,1.752732,-1.265763,-0.209128,1.706432,0.509081,-0.200540,-0.257648,0.023685,0.138988,0.443130,0.582677,-0.696082,-0.738528,-0.671822,0.710721,1.093609,1.258001,0.094472,0.618456,0.412917,1.082721,0.002357,0.635525,0.035163,1,1,1,0
3,1,0,0,0,0,0,0.134364,0.847434,-1.0,-1.000000,-1.000000,-1.000000,-2.160263,0.445070,-0.368192,0.210312,0.838155,-1.017779,2.181343,-0.352907,-0.039019,-0.690817,-0.894821,-0.783836,-0.591347,-0.696082,-0.738528,-0.671822,-0.420297,-0.253640,-0.122403,-0.921821,-0.861189,-1.566257,-1.881579,-1.853896,-1.986193,0.035163,0,0,0,0
4,0,0,1,0,0,0,-1.000000,-1.000000,-1.0,0.763775,-1.000000,-1.000000,0.354542,0.637739,1.558871,-1.288921,-0.318990,1.667101,0.432802,-0.195713,-0.346153,-0.009741,0.138988,0.443130,0.582677,-0.696082,-0.738528,-0.671822,0.710721,1.093609,1.258001,0.094413,0.618456,0.412917,0.790924,0.002357,0.377451,0.035163,1,1,0,0


In [33]:
initial_data.to_csv('data/entmoot_optimization/bo_initial_data.csv', index=False)

### 2. Train Bayesian optimizer

In [76]:
# Load initial data
initial_data = pd.read_csv('data/entmoot_optimization/bo_initial_data.csv')

In [77]:
# Drop good (>= "Okay") skew predictions as it is predicted way too often
initial_data = initial_data[((initial_data['an_vec_3'] != 1) | (initial_data['rating_1'] != 1))].reset_index(drop=True)

X_train = initial_data.iloc[:100, :-4]
X_train.replace(to_replace=-1, value=0, inplace=True)

y_train = initial_data.iloc[:100, -4:]

y_train = pd.Series(decode_classes(y_train.to_numpy()).flatten()) * -1

X_train = X_train.values.tolist()
y_train = y_train.values.tolist()

In [4]:
# Load surrogate model for function evaluations
func = SurrogateModelFNN()

In [79]:
# Fit optimizer to given initial data
start_time = time.time()
optimizer = entmoot_fit(dimensions=func.get_bounds(), x0=X_train, y0=y_train, random_state=73, verbose=0, base_estimator="RF", std_estimator="L1DDP")
print("--- %s seconds ---" % (time.time() - start_time))

# Save fitted optimizer
with open('models/entmoot_optimizer.pkl', 'wb') as output:
    pickle.dump(optimizer, output, pickle.HIGHEST_PROTOCOL)

------------------------------------------------------------
Fitted a model to observed evaluations of the objective.
--- 3.8621978759765625 seconds ---


In [5]:
# Show example of animation prediction

# Load optimizer
with open("models/entmoot_optimizer.pkl", "rb") as f:
    optimizer = pickle.load(f)

# Predict animations for new path
path_vector = [9.0731e-02,  9.1726e-01, -2.2292e-01,
         1.0261e+00, -1.0310e+00,  3.8478e-01,  4.4583e-01,  3.7610e+00,
         9.0707e-02, -1.5281e+00, -1.1364e+00, -1.0705e+00, -8.6565e-01,
        -7.8171e-01, -9.8579e-01, -8.6640e-01, -6.1583e-01, -3.6518e-01,
        -2.7900e-01, -7.6306e-01,  2.0780e+00,  4.8552e-01, -1.5957e-01,
        -2.7360e-03, -1.3878e+00,  9.0921e-02]

start_time = time.time()
opt_x, opt_y = entmoot_predict(optimizer, func, path_vector)
print("--- %s seconds ---" % (time.time() - start_time))

print(f'Predicted animation vector: {opt_x}')
print('-'*60)
print(f'Predicted animation score: {-opt_y}')

Academic license - for non-commercial use only - expires 2021-06-18
Using license file /Users/jani/gurobi.lic
--- 0.9895589351654053 seconds ---
Predicted animation vector: [0, 0, 0, 0, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.090731, 0.91726, -0.22292, 1.0261, -1.031, 0.38478, 0.44583, 3.761, 0.090707, -1.5281, -1.1364, -1.0705, -0.86565, -0.78171, -0.98579, -0.8664, -0.61583, -0.36518, -0.279, -0.76306, 2.078, 0.48552, -0.15957, -0.002736, -1.3878, 0.090921]
------------------------------------------------------------
Predicted animation score: 3


In [6]:
# Load test data for checking evaluation procedure
eval_data = pd.read_csv('data/entmoot_optimization/evaluation_data.csv')

In [8]:
path_vectors = eval_data[config.sm_features].values.tolist()[0:100]

In [9]:
# Predict animation vectors
an_vec_preds = []
score_preds = []
for i in range(len(path_vectors)):
    opt_x, opt_y = entmoot_predict(optimizer, func, path_vectors[i])
    an_vec_preds.append(opt_x)
    score_preds.append(-opt_y)
    if (i+1) % 10 == 0:
        print(f'Finished animation prediction for path {i+1}.')

Finished animation prediction for path 10.
Finished animation prediction for path 20.
Finished animation prediction for path 30.
Finished animation prediction for path 40.
Finished animation prediction for path 50.
Finished animation prediction for path 60.
Finished animation prediction for path 70.
Finished animation prediction for path 80.
Finished animation prediction for path 90.
Finished animation prediction for path 100.


In [10]:
np.mean(score_preds)

1.91

In [11]:
n_translate = np.sum([int(l[0]==1) for l in an_vec_preds])
n_scale = np.sum([int(l[1]==1) for l in an_vec_preds])
n_rotate = np.sum([int(l[2]==1) for l in an_vec_preds])
n_skew = np.sum([int(l[3]==1) for l in an_vec_preds])
n_fill = np.sum([int(l[4]==1) for l in an_vec_preds])
n_opacity = np.sum([int(l[5]==1) for l in an_vec_preds])

In [12]:
print(f'number of "translate" predictions: {n_translate}')
print(f'number of "scale" predictions: {n_scale}')
print(f'number of "rotate" predictions: {n_rotate}')
print(f'number of "skew" predictions: {n_skew}')
print(f'number of "fill" predictions: {n_fill}')
print(f'number of "opacity" predictions: {n_opacity}')

number of "translate" predictions: 0
number of "scale" predictions: 49
number of "rotate" predictions: 3
number of "skew" predictions: 0
number of "fill" predictions: 16
number of "opacity" predictions: 32
